In [1]:
from langchain.document_loaders import DirectoryLoader

directory = './data/'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

c:\Users\stefa\anaconda3\envs\py3aai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3

In [2]:
documents

[Document(page_content="Prestazioni occasionali: le novità 2023 della Legge di Bilancio\n\nCon la Circolare n. 6/2023 l’Inps illustra le nuove disposizioni introdo(cid:425)e dall’art. 1, c. 342 e 343, della L. 29 dicembre 2022, n. 197\n\nDi Laura Biarella Avvocato in Perugia Pubblicato il 01/02/2023 Con una Circolare 19 gennaio 2023, n. 6 (testo in calce) l’Inps ha fornito chiarimen(cid:415) in materia di prestazioni occasionali, con riferimento alle nuove disposizioni introdo(cid:425)e dall’art. 1, c. 342 e 343, della Legge di Bilancio per il 2023. Sommario La norma(cid:415)va Limi(cid:415) economici per l’accesso al Libre(cid:425)o Famiglia e al Contra(cid:425)o di prestazione occasionale Nuovo limite dimensionale degli u(cid:415)lizzatori del Contra(cid:425)o di prestazione occasionale Divieto di u(cid:415)lizzo del Contra(cid:425)o di prestazione occasionale per l’agricoltura La norma(cid:415)va L’art. 54-bis del d.l. n. 50/2017, introdo(cid:425)o, in sede di conversione, dalla L. 

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs


docs = split_docs(documents)
print(len(docs))

143


In [4]:
# # import openai
# from langchain.embeddings.openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model_name="ada")
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\Users\stefa\anaconda3\envs\py3aai\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\stefa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformers.json

In [5]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

In [6]:
query = "Cosa è una prestazione occazionale?"
matching_docs = db.similarity_search(query)

In [8]:
matching_docs[0]

Document(page_content='prestazione di giardinaggio, la riparazione di un PC, l’eﬀe(cid:425)uazione di ripe(cid:415)zioni scolas(cid:415)che, etc. Non vi sono più obblighi di legge su numero di giorni lavora(cid:415)vi e compenso percepito (con l’abrogazione del job’s act del 2015). Oggi la prestazione di lavoro autonomo occasionale è normata solo dall’art. 2222 del c.c., dalle circolari INPS, e dal DPR n. 633/72 per quanto riguarda gli obblighi di apertura della par(cid:415)ta IVA per le prestazioni di lavoro autonomo abituali e professionali.\n\nCome funziona la prestazione con ritenuta d’acconto?', metadata={'source': 'data\\file3.pdf'})

In [9]:
matching_docs = db.similarity_search_with_score(query, k=2)
matching_docs

[(Document(page_content='prestazione di giardinaggio, la riparazione di un PC, l’eﬀe(cid:425)uazione di ripe(cid:415)zioni scolas(cid:415)che, etc. Non vi sono più obblighi di legge su numero di giorni lavora(cid:415)vi e compenso percepito (con l’abrogazione del job’s act del 2015). Oggi la prestazione di lavoro autonomo occasionale è normata solo dall’art. 2222 del c.c., dalle circolari INPS, e dal DPR n. 633/72 per quanto riguarda gli obblighi di apertura della par(cid:415)ta IVA per le prestazioni di lavoro autonomo abituali e professionali.\n\nCome funziona la prestazione con ritenuta d’acconto?', metadata={'source': 'data\\file3.pdf'}),
  0.8333523273468018),
 (Document(page_content='oppure la durata complessiva della prestazione di 280 ore nell’arco dello stesso anno civile, salvo che per le pubbliche amministrazioni, il contra(cid:425)o si trasforma in un rapporto di lavoro a tempo pieno e indeterminato. In caso di violazione dell’obbligo di comunicazione all’INPS oppure delle 

In [10]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

In [11]:
vectordb.persist()

In [12]:
new_db = Chroma(persist_directory=persist_directory,
                embedding_function=embeddings)

In [13]:
matching_docs = new_db.similarity_search_with_score(query)
matching_docs[0]

(Document(page_content='prestazione di giardinaggio, la riparazione di un PC, l’eﬀe(cid:425)uazione di ripe(cid:415)zioni scolas(cid:415)che, etc. Non vi sono più obblighi di legge su numero di giorni lavora(cid:415)vi e compenso percepito (con l’abrogazione del job’s act del 2015). Oggi la prestazione di lavoro autonomo occasionale è normata solo dall’art. 2222 del c.c., dalle circolari INPS, e dal DPR n. 633/72 per quanto riguarda gli obblighi di apertura della par(cid:415)ta IVA per le prestazioni di lavoro autonomo abituali e professionali.\n\nCome funziona la prestazione con ritenuta d’acconto?', metadata={'source': 'data\\file3.pdf'}),
 0.8333523010931576)

In [20]:
import os
os.environ["OPENAI_API_KEY"] = "sk..Sx"

In [21]:
from langchain_openai import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [22]:
from langchain.chains.question_answering import load_qa_chain
# chain = load_qa_chain(llm, chain_type="stuff")
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

In [31]:
query = "cosa dice la Circolare 19 gennaio 2023, n. 6?"
matching_docs = db.similarity_search(query)
answer = chain.run(input_documents=matching_docs, question=query)
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Prestazioni occasionali: le novità 2023 della Legge di Bilancio

Con la Circolare n. 6/2023 l’Inps illustra le nuove disposizioni introdo(cid:425)e dall’art. 1, c. 342 e 343, della L. 29 dicembre 2022, n. 197

Il Decreto Legge 24 aprile 2017, n. 50 Apre in una nuova scheda, conver(cid:415)to con modiﬁcazioni in Legge 21 giugno 2017, n. 96, disciplina le prestazioni occasionali (art. 54 bis). In par(cid:415)colare, la norma(cid:415)va vigente amme(cid:425)e la possibilità di svolgere prestazioni di lavoro occasionale, intendendosi per tali le a(cid:427)vità lavora(cid:415)ve che danno luogo nel corso di un anno civile (1° gennaio – 31 dicembre di ciascun anno): a) per ciascun prestatore, con riferimen

"La Circolare 19 gennaio 2023, n. 6 (Circolare n. 6/2023) dell'Inps illustra le nuove disposizioni introdotte dall'art. 1, c. 342 e 343, della Legge 29 dicembre 2022, n. 197. Tuttavia, non è specificato nel testo quale sia il contenuto specifico di questa circolare. Quindi, non posso fornire ulteriori dettagli sulla Circolare n. 6/2023."

In [32]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(
    llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'La Circolare 19 gennaio 2023, n. 6 non viene menzionata nel testo fornito. Pertanto, non posso fornire informazioni specifiche su ciò che dice questa circolare.'